In [ ]:
//load the movies data from hdfs stored as movies
val rdd1=sc.textFile("movies")
// loaded data has 4 coloumns but we require only movie_id and movie name threrfore we map these 2 values out
val rdd2=rdd1.map(x=>{
          val fields=x.split("::")
          (fields(0),fields(1))
      })
// loading the ratings file from hdfs

val ratings=sc.textFile("ratings")
// rating file has many columns so but we have
// input (user_id,movie_id,rating,timestamp)
//output (movie_id,ratings)
val new_ratings=ratings.map(x=>{
          val fields=x.split("::")
          (fields(1),fields(2))
      })
// input (1127,4.5)
// output (1127,(4.5,1.0))

val rdd3=new_ratings.map(x=>{
     |     (x._1,(x._2.toFloat,1.0))
     |     
     | });

// input (1127,(4.5,1.0)),(1127,(5,1)),(1127,(3.5,1))
// output (1127,(13,3))
val aggregation=rdd3.reduceByKey((x,y)=>(x._1+y._1,x._2+y._2))
// can be done in the above step as optimization 
val rdd4=rdd3.map(x=>(x._1,(x._2._1.toFloat,x._2._2.toFloat)));
// input (1127,(13,3))
// output (only those movies which have count greater than 1000 have to be filtered out)
val rdd6=rdd5.filter(x=>x._2._2>1000)
// find the average rating  by sum(ratings)/numberofratings
val rdd7=rdd6.map(x=>(x._1,x._2._1/x._2._2))
// filter only those movies where ratings are greater than 4.5
val final_movies=rdd7.filter(x=>x._2>4.5)
// input (movie_id,ratings) and  (movie_id,names)
//output(movie_id,(name,ratings))


val joineed=final_movies.join(rdd2)
// input (movie_id,(movie_name,ratings))
//output (movie_name)
val top_movies=joineed.map(x=>x._2._2)

val sorted_top_movies=joineed.sortBy(x=>x._2._1)
sorted_top_movies.collect
